# Maximum Likelihood Estimation of a multivariate Gaussian model

The goal of this notebook is to use PyTorch to implement Gradient-based MLE for a multivariate Gaussian model.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

## Single 2D Gaussian component

Let's generate a some data by sampling a multivariate Gaussian with an arbitrary covariance matrix:

In [ ]:
rng = np.random.RandomState(42)
n_features = 2

mean = rng.randn(n_features)
mean

In [ ]:
h = rng.randn(n_features, n_features)
Cov = h @ h.T
Cov

In [ ]:
np.linalg.cholesky(np.linalg.inv(Cov))

In [ ]:
n_samples = 100
data = rng.multivariate_normal(mean, Cov, size=n_samples)
plt.scatter(data[:, 0], data[:, 1]);

Let's compute the MLE estimate from this data using the closed-form formula:

In [ ]:
mu_mle = data.mean(axis=0)
mu_mle

In [ ]:
Cov_mle = (data - mu_mle).T @ (data - mu_mle) / n_samples
Cov_mle

In [ ]:
np.linalg.cholesky(np.linalg.inv(Cov_mle))

## Parametrisation of a positive definite matrix


Let's parametrize the precision matrix `P` (inverse of a covariance matrix `C`) as follows:

- `P` has Cholesky decomposition `H`
- `H` is a lower triangular with a positive diagonal
- the log of the diagonal entry is stored in a vector of parameters named `d`
- the off diagonal elements of `H` are stored in the matrix of parameters named `W`

In [ ]:
import torch
from torch.autograd import Variable

mu = Variable(torch.zeros(n_features), requires_grad=True)
d = Variable(torch.ones(n_features), requires_grad=True)
W = Variable(torch.randn(n_features, n_features), requires_grad=True)
H = torch.diag(torch.exp(d)) + torch.tril(W, -1)
P = H @ H.transpose(1, 0)
P

Let's check that H is the actual Cholesky decomposition of P:

In [ ]:
H.data.numpy()

In [ ]:
np.linalg.cholesky(P.data.numpy())

`P` is positive semi-definite by construction (product of a matrix `H` by its transposed).

Because of we take the `exp` of `d` to build the diagonal elements of `H`, the determinant of `H` and therefore `P` is stricly positive.

`P` is therefore is positive definite, whatever the values the parameters in `d` and `W`. Because the Cholesky decomposition exists for any symmetric positive-definite  matrix and is unique and `exp` is a bijection from $\mathbb{R}$ to $\mathbb{R}^+$, this parametrization of the manifold of positive definite matrices is bijective.

In [ ]:
np.linalg.det(P.data.numpy())

In [ ]:
np.linalg.det(H.data.numpy()) ** 2

The determinant of `P` is cheap to compute from the `d` parameters directly:

In [ ]:
torch.prod(torch.exp(d.data) ** 2)

Let's use the above function to define the log-likelihood of a Gaussian model:

In [ ]:
from math import log


def loglik(X, mu, d, W):
    """Compute the average log-likelihood of samples
    
    X shape: (n_samples, n_features)
        data points
        
    mu: shape: (n_features,)
        parameters of the mean of the Gaussian.
    
    d: shape: (n_features,)
        parameters of the diagonal of the Cholesky factor of the
        precision matrix of the Gaussian.
        
    W: shape: (n_features, n_features)
        parameters of the off-diagonal of the Cholesky factor of the
        precision matrix of the Gaussian. The upper-diagonal elements
        are ignored.
    """
    H = torch.diag(torch.exp(d)) + torch.tril(W, -1)
    P = H @ H.transpose(1, 0)
    diff = X - mu
    quad_form = torch.sum(diff * (diff @ P), dim=1)
    return (-0.5 * log(2 * np.pi) + torch.sum(d) - 0.5 * quad_form)


def nll(X, mu, d, W):
    """Average negative log likelihood loss to minimize"""
    return -torch.mean(loglik(X, mu, d, W))

In [ ]:
X = Variable(torch.FloatTensor(data))
loss = nll(X, mu, d, W)
loss

In [ ]:
loss.backward([torch.ones(1)])

In [ ]:
mu.grad

In [ ]:
d.grad

In [ ]:
W.grad

In [ ]:
H_mle = torch.FloatTensor(np.linalg.cholesky(np.linalg.inv(Cov_mle)))
d_mle = Variable(torch.log(torch.diag(H_mle)))
W_mle = Variable(torch.tril(H_mle, -1))
nll(X, Variable(torch.FloatTensor(mu_mle)), d_mle, W_mle)

In [ ]:
learning_rate = 1e-1
optimizer = torch.optim.Adam([mu, d, W], lr=learning_rate)
for t in range(2000):
    # Compute and print loss.
    loss = nll(X, mu, d, W)
    if t % 100 == 0:
        print(t, loss.data[0])
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
mu_mle

In [ ]:
mu

In [ ]:
d_mle

In [ ]:
d

In [ ]:
W * Variable(torch.tril(torch.ones(n_features, n_features), -1))

In [ ]:
W_mle

## Mixture of 2D Gaussian components

Let's generate some ground truth Gaussian Mixture Model with 3 components.

In [ ]:
rng = np.random.RandomState(42)
n_features = 2
n_components = 3

true_means = [rng.randn(n_features) * 3 for _ in range(n_components)]
true_means

In [ ]:
true_covariances = []
for _ in range(n_components):
    h = rng.randn(n_features, n_features)
    true_covariances.append(h @ h.T)

Let's generate some data from the ground truth model:

In [ ]:
from sklearn.utils import shuffle

n_samples_per_component = 100

samples = []
component_ids = []

for i, mean, Cov in zip(range(n_components), true_means, true_covariances):
    data = rng.multivariate_normal(mean, Cov, size=n_samples_per_component)
    samples.append(data)
    component_ids.append(i * np.ones(n_samples_per_component, dtype=np.int32))
    plt.scatter(data[:, 0], data[:, 1])
    
data = np.vstack(samples)
component_ids = np.concatenate(component_ids)

data, component_ids = shuffle(data, component_ids, random_state=0)

There is no closed form formula for the MLE. Let's use the scikit-learn implementation of the EM algorithm instead:

In [ ]:
from sklearn.mixture import GaussianMixture


gmm_em = GaussianMixture(n_components=3, random_state=0).fit(data)

Average loglikelihood of the data under the EM-MLE model (higher is better):

In [ ]:
gmm_em.score(data)

In [ ]:
gmm_em.means_

In [ ]:
true_means

In [ ]:
gmm_em.weights_

In [ ]:
gmm_em.covariances_

In [ ]:
np.asarray(true_covariances)

Let's find the MLE by gradient descent. First we need a helper function to compute the log of the sum of likelihoods of the components in a numerically stable way:

In [ ]:
def logsumexp(data, dim=0):
    """Numerically stable log sum exp"""
    max_score, _ = torch.max(data, dim)
    if dim == 0:
        max_score_bcast = max_score
    elif dim == 1:
        max_score_bcast = max_score.view(-1, 1)
    else:
        raise NotImplemented("logsumexp with dim=%d is not supported" % dim)
    return max_score + torch.log(torch.sum(torch.exp(data - max_score_bcast), dim))


test_data = torch.randn(3, 4)
logsumexp(test_data, 0)

In [ ]:
torch.log(torch.sum(torch.exp(test_data), 0))

In [ ]:
X = Variable(torch.FloatTensor(data))


logsotfmax = torch.nn.LogSoftmax()
weights = Variable(torch.ones(1, n_components), requires_grad=True)

means = []
prec_diags = []
prec_off_diags = []
for i in range(n_components):
    mu = Variable(torch.randn(n_features), requires_grad=True)
    means.append(mu)
    d = Variable(torch.ones(n_features), requires_grad=True)
    prec_diags.append(d)
    W = Variable(torch.randn(n_features, n_features), requires_grad=True)
    prec_off_diags.append(W)


def mixture_nll(X, weights, means, prec_diags, prec_off_diags):
    log_normalized_weights = logsotfmax(weights).transpose(1, 0)
    logliks = []
    for mu, d, W in zip(means, prec_diags, prec_off_diags):
        logliks.append(loglik(X, mu, d, W))
    
    logliks = torch.cat(logliks).view(n_components, -1)
    return torch.mean(-logsumexp(logliks + log_normalized_weights, dim=0))

In [ ]:
mixture_nll(X, weights, means, prec_diags, prec_off_diags)

In [ ]:
params = [weights]
params.extend(means)
params.extend(prec_diags)
params.extend(prec_off_diags)

In [ ]:
weights

In [ ]:
learning_rate = 0.05
optimizer = torch.optim.Adam(params, lr=learning_rate)
best_loss = np.inf
for t in range(5000):
    loss = mixture_nll(X, weights, means, prec_diags, prec_off_diags)
    if t % 100 == 0:
        print(t, loss.data[0])
    if loss.data.numpy() < best_loss:
        best_loss = loss.data.numpy()
    else:
        print(t, loss.data[0], 'converged!')
        break
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
[m.data.numpy() for m in means]

In [ ]:
gmm_em.means_

In [ ]:
true_means

In [ ]:
softmax = torch.nn.Softmax()
softmax(weights).view(-1).data.numpy()

In [ ]:
gmm_em.weights_